In [ ]:
# !pip install numpy pandas scipy 

In [1]:
import numpy as np
from scipy.linalg import lstsq
import random
import pandas as pd
import os

type_converter = {i: lambda x: int(x, 16) for i in range(16)}
LAB_PATH = "/scratch/net4/HOS/Traces"
MAX_ROWS = 30000 #Total Number of rows to be selected
ROUND1_START = 20000
ROUND1_END = 25000
SCALE = 3
m_off = 0
AES_SBOX = [0x6, 0xB, 0x5, 0x4, 0x2, 0xE, 0x7, 0xA,0x9, 0xD, 0xF, 0xC, 0x3, 0x1, 0x0, 0x8]
random_plaintexts = np.loadtxt(os.path.join(LAB_PATH,"plaintexts.txt"), dtype=np.uint8, max_rows=MAX_ROWS, converters=type_converter)
random_traces = np.loadtxt(os.path.join(LAB_PATH,"traces.txt"), max_rows=MAX_ROWS)
unscaled_traces = random_traces[:,ROUND1_START:ROUND1_END]

scaled_traces = ((unscaled_traces / 127) * 4) * SCALE + m_off

print("unscaled traces shape: ", unscaled_traces.shape)
print("scaled traces shape: ", scaled_traces.shape)

unscaled traces shape:  (30000, 5000)
scaled traces shape:  (30000, 5000)


In [11]:
import numpy as np
from scipy.linalg import lstsq, pinv
import random

class BasisFunctions:
    """Handles the precomputation of the basis matrix G for a 4-bit S-Box."""
    def __init__(self, sbox, basis_type='bits'):
        self.sbox = sbox
        if basis_type in ["hw", "lsb"]:
            self.num_basis = 2
        else:
            self.num_basis = 5
        self.basis_type = basis_type
        self.G = self._build_basis_matrix()

    def _build_basis_matrix(self):
        """Precomputes the basis matrix G for all phi values (0-15) based on actual bit value and HW"""
        G = np.zeros((16, self.num_basis), dtype=float)
        for phi in range(16):
            sbox_output = self.sbox[phi]
            if(self.basis_type == 'bits'):
                G[phi] = [1.0]+[float(i) for i in format(sbox_output, '04b')]
            elif(self.basis_type == 'hw_bits'):
                G[phi] = [1.0] + [float(i) for i in format(bin(sbox_output).count('1'),'04b')]
            elif(self.basis_type == 'hw'):
                G[phi] = [1.0] + [float(bin(sbox_output).count('1'))]
            elif(self.basis_type == 'lsb'):
                G[phi] = [1.0, float(sbox_output & 1)]
        return G

class Profiling:
    """Manages the profiling phase: beta estimation, time point selection, and covariance estimation."""
    def __init__(self, basis_functions):
        self.basis_functions = basis_functions
        self.betas = None
        self.ts = None
        self.cov = None

    def estimate_betas(self, traces_prof, x_prof, k_b, start_time, end_time):
        phi_prof = np.bitwise_xor(x_prof, k_b) & 0x0F
        G_prof = self.basis_functions.G[phi_prof]  # shape (N1, num_basis)
        I = traces_prof[:, start_time:end_time]  # shape (N1, segment_length)
        G_pinv = pinv(G_prof)
        betas = G_pinv @ I    # Shape (num_basis, segment_length)
        self.betas = betas.T  # Match old shape (segment_length, num_basis)
        self.start_time = start_time
        return self.betas

    def select_time_points(self,betas, tau, selection_mode='S2', traces_prof=None):
        """Selects relevant time points ts based on the norm of data-dependent betas."""
        norm_b = np.linalg.norm(betas[:, 1:], axis=1)  # Exclude constant term
        if selection_mode == 'S3':
            self.ts = [np.argmax(norm_b)]  #Single max peak
        elif selection_mode == 'S2':
            self.ts = [int(x) for x in np.argsort(norm_b)[-5:]]  # Top 10 peaks
        elif selection_mode == 'S6':
            self.ts = np.argsort(norm_b)[-21:]  #Top 21 peaks
        elif selection_mode == 'S1':
            self.ts = np.where(norm_b >= tau)[0]  #Threshold-based
        elif selection_mode in ['S4', 'S5']:
            if traces_prof is None:
                raise ValueError("traces_prof required for S4/S5 selection modes")
            var_t = np.var(traces_prof, axis=0)
            mask = (norm_b >= tau) & (norm_b > var_t)
            self.ts = np.where(mask)[0]
            if selection_mode == 'S5':
                extra_mask = (norm_b >= tau / 2) & ~mask
                self.ts = np.concatenate((self.ts, np.where(extra_mask)[0]))
        else:
            raise ValueError(f"Unknown selection_mode: {selection_mode}")
        return self.ts
    
    def detect_noise(self, traces_prof, x_prof, k_b):
        """Detect noise levels using residuals after beta fitting."""
        if self.betas is None:
            raise ValueError("Run estimate_betas first")
        
        # Compute predicted h_t for all t in segment
        phi_prof = np.bitwise_xor(x_prof, k_b) & 0x0F
        G_prof = self.basis_functions.G[phi_prof]  # N1 x 5
        h_pred = np.dot(G_prof, self.betas.T)  # N1 x segment_length
        # Residuals = observed - predicted
        segment_traces = traces_prof[:, self.start_time:self.start_time + len(self.betas)]
        residuals = segment_traces - h_pred
        # Noise metrics
        noise_var = np.var(residuals)  # Overall var(R_t)
        signal_var = np.var(h_pred)    # var(h_t)
        snr = signal_var / noise_var if noise_var > 0 else float('inf')
        # Per-time variance
        noise_var_per_t = np.var(residuals, axis=0)  # segment_length x 1
        return snr, noise_var, noise_var_per_t


class KeyExtraction:
    """Handles key extraction using minimum or maximum likelihood principles."""
    def __init__(self, profiling):
        self.profiling = profiling
        self.basis_functions = profiling.basis_functions
        self.betas = profiling.betas
        self.ts = profiling.ts
        self.cov = profiling.cov

    def extract_key(self, traces_attack, x_attack,betas, ts,  N3, method='minimum'):
        """Extracts the subkey using the specified method."""
        extracted_key = {i: 0 for i in range(16)}
        G = self.basis_functions.G
        betas_ts = betas[ts]
        if method == 'minimum':
            min_diff = float('inf')
            best_k = None
            for k_prime in range(16):
                diff = 0.0
                for j in range(N3):
                    phi_j = np.bitwise_xor(x_attack[j], k_prime) & 0x0F
                    h_j = betas_ts @ G[phi_j]
                    i_j = traces_attack[j, ts]
                    diff += np.sum((i_j - h_j) ** 2)
                avg_diff = diff / N3
                if avg_diff < min_diff:
                    min_diff = avg_diff
                    best_k = k_prime
                    extracted_key[k_prime] = extracted_key[k_prime]+1
            return best_k, extracted_key
        else:
            raise ValueError(f"Unknown method: {method}")


    def compute_success_rate(self, traces_attack, x_attack,betas,timepoints, correct_k, sampled=50, num_trials=10, method='minimum'):
        """Computes the success rate over multiple trials with random subsets."""
        num_attack = len(x_attack)
        success = 0
        for _ in range(num_trials):
            indices = random.sample(range(num_attack), sampled)
            traces_sub = traces_attack[indices]
            x_sub = x_attack[indices]
            extracted_k = self.extract_key(traces_sub, x_sub,betas, timepoints, sampled, method)
            if extracted_k == correct_k:
                success += 1
        return (success / num_trials) * 100
    
def create_range(start_time=0, end_time=5000, increments=(5, 10), widths=(10, 20)):
    ranges = set()
    widths = tuple(sorted(widths))
    min_w = widths[0]

    for inc in increments:
        s = start_time
        while s + min_w <= end_time:
            for w in widths:
                end = s + w
                if end > end_time:
                    break
                ranges.add((s, end))
            s += inc
    return sorted(ranges)

In [15]:
# import SDSCA as sd
total_prof = 20000
key = np.array([15,14,13,12,11,10,9,8,7,6,5,4,3,2,1,0], dtype=np.uint8)
ATTACK_START = 25000
ATTACK_END = 30000
TOTAL_ATTACK = ATTACK_END - ATTACK_START
PROF_START = 0
PROF_END = 5000
correct_count = 0
for byte_idx in range(16):
    plaintexts_all = random_plaintexts[:, byte_idx] & 0x0F  # Lower 4 bits
    k_b = key[byte_idx] & 0x0F
    traces_prof = scaled_traces[:total_prof]
    plaintexts_prof = plaintexts_all[:total_prof]
    AES_SBOX = [0x6, 0xB, 0x5, 0x4, 0x2, 0xE, 0x7, 0xA,0x9, 0xD, 0xF, 0xC, 0x3, 0x1, 0x0, 0x8]
    bf = BasisFunctions(AES_SBOX, basis_type="hw")
    prof = Profiling(bf)
    betas = prof.estimate_betas(traces_prof, plaintexts_prof, k_b,PROF_START, PROF_END)
    timepoints = prof.select_time_points(betas, 0.1, selection_mode="S2")
    traces_attack = scaled_traces[ATTACK_START:ATTACK_END]
    plaintexts_attack = plaintexts_all[ATTACK_START:ATTACK_END]
    ke = KeyExtraction(prof)
    extracted_k, exp_keys = ke.extract_key(traces_attack, plaintexts_attack, betas, timepoints, TOTAL_ATTACK, method="minimum")
    filtered = {k: v for k, v in exp_keys.items() if v}
    if key[byte_idx] in filtered.keys():
        correct_count += 1
    print("Extracted key = ", extracted_k,filtered, "correct key = ", key[byte_idx])
f"Total correct bytes: {correct_count}/{16}"

Extracted key =  15 {0: 1, 2: 1, 3: 1, 15: 1} correct key =  15
Extracted key =  14 {0: 1, 3: 1, 14: 1} correct key =  14
Extracted key =  14 {0: 1, 1: 1, 2: 1, 6: 1, 13: 1, 14: 1} correct key =  13
Extracted key =  12 {0: 1, 12: 1} correct key =  12
Extracted key =  11 {0: 1, 1: 1, 7: 1, 8: 1, 11: 1} correct key =  11
Extracted key =  10 {0: 1, 1: 1, 6: 1, 10: 1} correct key =  10
Extracted key =  9 {0: 1, 3: 1, 9: 1} correct key =  9
Extracted key =  8 {0: 1, 1: 1, 2: 1, 8: 1} correct key =  8
Extracted key =  7 {0: 1, 4: 1, 6: 1, 7: 1} correct key =  7
Extracted key =  6 {0: 1, 4: 1, 5: 1, 6: 1} correct key =  6
Extracted key =  5 {0: 1, 1: 1, 2: 1, 4: 1, 5: 1} correct key =  5
Extracted key =  4 {0: 1, 3: 1, 4: 1} correct key =  4
Extracted key =  3 {0: 1, 2: 1, 3: 1} correct key =  3
Extracted key =  2 {0: 1, 1: 1, 2: 1} correct key =  2
Extracted key =  1 {0: 1, 1: 1} correct key =  1
Extracted key =  0 {0: 1} correct key =  0


'Total correct bytes: 16/16'